##Librerías varias que utilizaremos.

In [0]:
import requests, six
import lxml.html as lh
from itertools import cycle, islice
import pandas as pd

##Creamos los dataset con los datos del misal

In [0]:
url = 'https://www.minsal.cl/nuevo-coronavirus-2019-ncov/casos-confirmados-en-chile-covid-19/'

In [18]:
#Para realizar una solicitud HTTP
page = requests.get(url)

#Almacenar los contenidos de la web.
doc = lh.fromstring(page.content)

#Los datos que esten entre <tr>...</tr>
tr_elements = doc.xpath('//tr')

#Aquí checamos si las filas tienen el mismo largo
[len(T) for T in tr_elements[2:20]]

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]

In [19]:
#Creamos una lista vacía.
col=[]
i=0

#Guardamos el encabezado y una lista vacia (primera tabla).
for t in tr_elements[2]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

#print(col)
casosR = []
i=0
#Guardamos el encabezado y una lista vacía (segunda tabla).
for u in tr_elements[20]:
    i+=1
    name_2=u.text_content()
    print('%d:"%s"'%(i,name_2))
    if i > 1:
      name_2 = int(name_2)
      casosR[0][1].append(name_2)
      break
    casosR.append((name_2,[]))
print(casosR)
#El segundo for lo usamos para obtener los datos de la segunda tabla, ya que el largo de la fila de la segunda tabla es distinto.

1:"Región"
2:"Casos nuevos"
3:"Casos totales"
4:"% Casos totales**"
5:"Fallecidos"
1:"Casos recuperados a nivel nacional"
2:"75"
[('Casos recuperados a nivel nacional', [75])]


In [0]:
for j in range(3,len(tr_elements)):

    T=tr_elements[j]
    
    #Esto para que no guarde filas que tengas menos o más de 5 columnas.
    if len(T)!=5:
        break
  
    i=0
    
    #Recorremos cada elemento que tenga la fila.
    for t in T.iterchildren():
        data=t.text_content() 
        #Compruebe si la fila está vacía.
        if i>0:
        #Convierte cualquier valor numérico a enteros.
            try:
                data=int(data)
            except:
                pass
        #Agrega el valor a la lista vacía en la columna i.
        col[i][1].append(data)
        i+=1

In [21]:
[len(C) for (title,C) in col]

[17, 17, 17, 17, 17]

##Creamos el primer dataset con los valores de la página.

In [22]:
Dict={title:column for (title,column) in col}
import pandas as pd
df=pd.DataFrame(Dict)
df.head()

,Región,Casos nuevos,Casos totales,% Casos totales**,Fallecidos
0,Arica y Parinacota,1,4,0 %,0
1,Tarapacá,1,6,0 %,0
2,Antofagasta,2,27,1 %,0
3,Atacama,1,2,0 %,0
4,Coquimbo,3,18,1 %,0


##Creamos el segundo dataset con los datos totales de cada columna

Esto es para tener ordenada la tabla y para que luego sea más fácil la manipulación de los datos.

In [0]:
Dic_2 = {title:column for (title,column) in casosR}

In [24]:
import datetime
from datetime import date

fw = pd.DataFrame(Dic_2)
R=df.iloc[16:,4:5].values
X=df.iloc[16:,1:2].values
Y=float(df.iloc[16:,2:3].values)
Y = int(Y*1000)
dia = str(date.today())
#print(R)
#print(X)
#print(Y)
fw = fw.assign(Casos_fallecidos=R)
fw = fw.assign(Casos_totales_dia=Y)
fw = fw.assign(Casos_totales_nuevos=X)
fw = fw.assign(Fecha=dia)
fw.head()

,Casos recuperados a nivel nacional,Casos_fallecidos,Casos_totales_dia,Casos_totales_nuevos,Fecha
0,75,7,2139,230,2020-03-29


##Por último creamos y actualizamos los archivos CSV

In [25]:
import os
import csv 
from google.colab import drive

drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/COVID-19 Chile (Coronavirus)/Database (CSV)') #En donde vamos a guardar los archivos CSV. 

#print(date.today())
df.drop(df.index[[16]],inplace=True)#Esto es para borrar los datos totales (ultima fila) de la primera tabla, porque recuerden que los vamos a guardar en la segunda.
df.to_csv('Casos_COVID-19_Chile_'+dia+'.csv', header=True, index=False) #Creamos el primer archivo con la primera tabla.
#fw.to_csv('Resumen_COVID-19_Chile_'+dia+'.csv', header=True, index=False) #Para crear todos los días un csv distinto con los datos de la segunda tabla.

#Aqui nos evitamos crear un csv para cada dia y tan solo actualizamos todos los dias el que ya existe (segunda tabla).
with open(r'Resumen_COVID-19_Chile_.csv', 'a') as f:
    writer = csv.writer(f)
    for row in fw.iloc[:,:].values:
      writer.writerow(row)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
